<a href="https://colab.research.google.com/github/bennsamuel/Pneumonia-Project/blob/master/parameter_augment_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you want to enable auto-save.  
-> Copy to drive (in navbar) 
-> creates copy in your google drive. that copy has to be manually commited to github.

CATEGORIES = ["NORMAL","BACTERIA", "VIRUS"]
0, 1, 2

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive/Group 77 - Pneumonia/pickle/"

data.pickle  test_data.pickle


In [3]:
IMG_SIZE = 224
NUM_CLASSES = 3
CATEGORIES = ["NORMAL","BACTERIA", "VIRUS"]
AUG_PATH='aug_images/'

%cd /content/

/content


In [0]:
#load dataset --> method baseline!
import pickle

data = pickle.load(open("/content/drive/My Drive/Group 77 - Pneumonia/pickle/data.pickle", "rb"))

np.array.reshape()

![alt text](https://www.w3resource.com/w3r_images/numpy-manipulation-reshape-function-image-a.png)

**Data structure**: 

[[numpy_array([[features]]), label],

 [numpy_array([[features]]), label],

 [numpy_array([[features]]), label],

...]

----------------------------------
No y target data because the array would look like: 

y_normal = [0,0,0,0,0,0...] 

y_bacteria = [1,1,1,1,1,1...] 

y_virus = [2,2,2,2,2,2...] 

add target label add the end. normal=0, bacteria=1, virus=2

In [0]:
import numpy as np

X_normal = []
X_bacteria = []
X_virus = []
y_normal = []
y_bacteria = []
y_virus = []

for features, label in data:
    if label == 0:
        X_normal.append(features)
        y_normal.append(0)
    elif label == 1:
        X_bacteria.append(features)
        y_bacteria.append(1)
    elif label == 2:
        X_virus.append(features)
        y_virus.append(2)
    else:
        raise Exception('Something went wrong')

X_normal = np.array(X_normal).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_bacteria = np.array(X_bacteria).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_virus = np.array(X_virus).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

**Which augmentations techniques to use**

While performing augmentation one must be sure to not use the wrong techniques for the domain you're working on. In our case this means that we for example shouldn't just flip the image, given the position of the other organs is important in the medical doamin.

After some thought i think the most valuable augmentation techniques for our problem will be **shifting, tilting (5°) and zooming of the image**.

In [0]:
import os
import shutil

#Determining which categories has the most pictures, to augment the other classes to that number 
num_of_most_occuring_class = max(len(X_normal),len(X_bacteria),len(X_virus))

Import of the necesarry librarys


In [7]:
from matplotlib import pyplot
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

Initialising the ImageDataGenerator, which will take take the data we will fit it to randomly perform the actions we set in it's parameters

Function that is used to perform the augmentation on the different categories

In [0]:
import cv2

def augment_smaller_classes(data_generator_with_aug, X_data, y_data, num_images_to_generate, name, prefix):
    print("       Category: " + name)
    if num_images_to_generate == 0:
          print("       No images have to be created")
          return
    # fitting the  ImageDataGenerator to the passed in data
    data_generator_with_aug.fit(X_data)
    print("       Images to create: " + str(num_images_to_generate))
    #For loop to create pictures
    i = 0
    for batch in data_generator_with_aug.flow(
        X_data,
        y_data,
        batch_size=1,
        save_to_dir=(AUG_PATH+name),
        save_prefix=prefix,
        save_format='jpeg'):
        i += 1
        if i > num_images_to_generate-1:
            print("       " + str(num_images_to_generate) + " Images created")
            break 

AUG_DIR = "/content/aug_images/"
CATEGORIES = ["NORMAL","BACTERIA", "VIRUS"]
def reload_created_data(aug_data):
        for category in CATEGORIES:
            path = os.path.join(AUG_DIR, category)
            #index: NORMAL --> 0, BACTERIA --> 1, VIRUS --> 2
            class_num = CATEGORIES.index(category)
            for img in os.listdir(path):
                try:
                    img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                    aug_data.append([new_array, class_num])
                except Exception as e:
                    print(e)

In [0]:
#print(num_of_most_occuring_class)

Apply function to the different categories, thus creating the needed additional 

In [0]:
import pickle


def parameter_study(WIDTH_SHIFT, HEIGHT_SHIFT, ZOOM, ROTATION, MIRROR_V, MIRROR_H):
    print("aug:{}_{}_{}_{}_{}_{} started".format(WIDTH_SHIFT,HEIGHT_SHIFT,ZOOM,ROTATION,MIRROR_V,MIRROR_H))
    #deleteing old folders if they already exist
    shutil.rmtree('/content/aug_images/', ignore_errors=True)
    #creating subfolders for all casesDIR = '/tmp'
    os.makedirs(AUG_PATH+'NORMAL')
    os.makedirs(AUG_PATH+'BACTERIA')
    os.makedirs(AUG_PATH+'VIRUS')
    path, dirs, files = next(os.walk("/content/aug_images/NORMAL"))
    file_count = len(files)
    print("   NORMAL" + str(file_count))
    path, dirs, files = next(os.walk("/content/aug_images/BACTERIA"))
    file_count = len(files)
    print("   BACTERIA" + str(file_count))
    path, dirs, files = next(os.walk("/content/aug_images/VIRUS"))
    file_count = len(files)
    print("   VIRUS" + str(file_count))
    data_generator_with_aug = ImageDataGenerator(
                                   width_shift_range = WIDTH_SHIFT, #allows a 20% shift to the left or the right
                                   height_shift_range = HEIGHT_SHIFT, #allows a 20% shift up or bottom
                                   zoom_range= ZOOM, #allows to zoom in 20%, meaning ther will always be 80% of the original picture
                                   rotation_range=ROTATION, #allows tilting of theimage to 5° in both directions
                                   vertical_flip=MIRROR_V,
                                   horizontal_flip=MIRROR_H,
                                   fill_mode='nearest'# method used for filling empty space that is created while shifting
                          )
    augment_smaller_classes(data_generator_with_aug, X_normal, y_normal, (num_of_most_occuring_class-len(X_normal)), 'NORMAL', 'normal')
    augment_smaller_classes(data_generator_with_aug, X_bacteria, y_bacteria, (num_of_most_occuring_class-len(X_bacteria)), 'BACTERIA', 'bacteria')
    augment_smaller_classes(data_generator_with_aug, X_virus, y_virus, (num_of_most_occuring_class-len(X_virus)), 'VIRUS', 'virus')
    
    aug_data = []
    reload_created_data(aug_data)
    combined_data = data + aug_data

    # save data set --> method baseline data
    # DON'T RENAME TO DATA.pickle WILL OVERWRITE OUR ORIGINAL DATA
    # Naming format: |width_shift_range|_|height_shift_range|_|zoom_range|_|rotation_range|_|vertical_flip|_|horizontal_flip|
    pickle_out = open("/content/drive/My Drive/Group 77 - Pneumonia/pickle_augmented/augmented-{}_{}_{}_{}_{}_{}.pickle".format(WIDTH_SHIFT,HEIGHT_SHIFT,ZOOM,ROTATION,MIRROR_V,MIRROR_H), "wb")
    pickle.dump(combined_data, pickle_out)
    pickle_out.close()
    print("aug:{}_{}_{}_{}_{}_{} ended".format(WIDTH_SHIFT,HEIGHT_SHIFT,ZOOM,ROTATION,MIRROR_V,MIRROR_H))



Parameter Study

In [12]:

for width_shift in range(0, 22, 2):
    for height_shift in range(0, 22, 2):
      for zoom in range(0, 22, 2):
        for rotation in range(0, 22, 2):
          for mirror_v in range(2):
            for mirror_h in range(2):
              mir_v = False if mirror_v == 0 else True
              mir_h = False if mirror_h == 0 else True
              parameter_study(width_shift, height_shift, float(zoom)/100, rotation, mir_v, mir_h)


                    


aug:0_0_0.0_0_False_False started
   NORMAL0
   BACTERIA0
   VIRUS0
       Category: NORMAL
       Images to create: 1189


KeyboardInterrupt: ignored

In [0]:
## checking the length of the data
#print(len(data))
#print(len(aug_data))

Checking if the data is ballanced now


In [0]:
#print(len(combined_data))
#num_normal = 0
#num_bacteria = 0
#num_virus = 0

#for features, label in combined_data:
#    if label == 0:
#        num_normal += 1
#    elif label == 1:
#        num_bacteria += 1
#    elif label == 2:
#        num_virus += 1
#    else:
#        raise Exception('Something went wrong')

#print("Images for no pneumonia: " + str(num_normal))
#print("Images for bacterial pneumonia: " + str(num_bacteria))
#print("Images for viral pneumonia: " + str(num_virus))